In [1]:
import cv2
import torch
from ultralytics import YOLO

from common.yolo import yolo_converter as converter
from common.yolo import nms
from common.tracker.SORT import SortTracker
from common.yolo.schema_loader import SchemaLoader
from common.yolo.drawer import Drawer

### Load the model and video and tracking the people

In [2]:
# Load the YOLOv8n-Pose model
model = YOLO("/opt/models/yolo/yolo11m-pose.pt")

# Test whether the GPU device is available
DEVICE = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'

# Move the model to the device
model.to(DEVICE)

# Create a drawer
schema = SchemaLoader()
drawer = Drawer(schema)

# Open the video file
cap = cv2.VideoCapture("/opt/videos/raining_street_02.mp4")
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Define the screen and window dimensions
window_width = 1024  # Window width as specified in cv2.resize()
window_height = 800  # Window height as specified in cv2.resize()

# Create the SORT tracker
tracker = SortTracker(max_age=10, min_hits=1, iou_threshold=0.5, max_objects=100)

# Set up the display window at (0, 0) position in the top-left corner
cv2.namedWindow("Frame", cv2.WINDOW_NORMAL)

# Move window to the top-left corner
cv2.moveWindow("Frame", 0, 0)

# Set the window size to the specified width and height
cv2.resizeWindow("Frame", window_width, window_height)

# Process the video
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Resize the frame
    frame = cv2.resize(frame, (460, 460))

    # Perform object detection
    results = model(frame, verbose=False)[0] # Get the first result

    # Process the pose
    poses = converter.results_to_pose_list(results)

    # Apply NMS to the poses
    poses = nms.nms_for_yolo_pose(poses)

    # Convert the list of poses to a numpy array
    poses_array = converter.pose_list_to_numpy(poses)

    # Perform object tracking
    tracked_objects = tracker.update(poses_array)

    # Convert tracked_objects to list
    sorted_list = converter.numpy_to_sorted_pose_list(tracked_objects)

    # Draw the bboxes without labels
    frame = drawer.draw_skeletons(frame, sorted_list, bbox_style="chromatic", show_pts=False, show_skeletons=False)

    # Display the frame
    frame = cv2.resize(frame, (window_width, window_height))
    cv2.imshow("Frame", frame)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# Release the video capture object and close the window
cap.release()
cv2.destroyAllWindows()

### Load the model and video and tracking the objects

In [3]:
# Load the YOLOv8n-Pose model
model = YOLO("/opt/models/yolo/yolov8n.pt")

# Test whether the GPU device is available
DEVICE = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'

# Move the model to the device
model.to(DEVICE)

# Create a drawer
schema = SchemaLoader()
drawer = Drawer(schema)

# Open the video file
cap = cv2.VideoCapture("/opt/videos/pedestrain_01.mp4")
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Define the screen and window dimensions
window_width = 1024  # Window width as specified in cv2.resize()
window_height = 800  # Window height as specified in cv2.resize()

# Create the SORT tracker
tracker = SortTracker(max_age=10, min_hits=1, iou_threshold=0.5, max_objects=100)

# Set up the display window at (0, 0) position in the top-left corner
cv2.namedWindow("Frame", cv2.WINDOW_NORMAL)

# Move window to the top-left corner
cv2.moveWindow("Frame", 0, 0)

# Set the window size to the specified width and height
cv2.resizeWindow("Frame", window_width, window_height)

# Process the video
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Resize the frame
    frame = cv2.resize(frame, (460, 460))

    # Perform object detection
    results = model(frame, verbose=False)[0] # Get the first result

    # Process the objects
    objects = converter.results_to_yolo_list(results)

    # Apply NMS to the objects
    objects = nms.nms_for_yolo(objects)

    # Convert the list of objects to a numpy array
    objects_array = converter.yolo_list_to_numpy(objects)

    # Perform object tracking
    tracked_objects = tracker.update(objects_array)

    # Convert tracked_objects to list
    sorted_list = converter.numpy_to_sorted_yolo_list(tracked_objects)

    # Draw the bboxes without labels
    frame = drawer.draw_objects(frame, sorted_list, bbox_style="chromatic")

    # Display the frame
    frame = cv2.resize(frame, (window_width, window_height))
    cv2.imshow("Frame", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close the window
cap.release()
cv2.destroyAllWindows()